In [1]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from scipy.stats import norm, skew #加载一些统计的库

In [2]:
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

In [3]:
def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    return data

In [4]:
data = pd.read_table('train.txt',sep=' ',index_col=False)
data2 = pd.read_table('test.txt',sep=' ',index_col=False) 
data3 = pd.read_table('test1.txt',sep=' ',index_col=False) 

In [5]:
data = pre_process(data)
data2 = pre_process(data2)
data3 = pre_process(data3)

item_category_list_ing
item_property_list_ing
item_category_list_ing
item_property_list_ing
item_category_list_ing
item_property_list_ing


## 用户对于一个类别的商品点击

In [6]:
data[['user_id','category_1','is_trade']].groupby(['user_id','category_1']).sum()

is_trade
user_id             category_1                   
1523553916360       1909641874861640857         0
                    453525480366550911          0
                    6693726201323251689         0
5742370312099       2871729383671301763         0
6731361713351       8841625760168847319         0
8455785307263       8841625760168847319         0
9451887501191       768579787521575291          0
12217660235260      1852600517265062354         0
                    5685690139879409547         0
15213145006378      453525480366550911          0
                    768579787521575291          0
22634503310052      1909641874861640857         0
                    1916390345133212703         0
25684305694756      5685690139879409547         0
38014406741475      2871729383671301763         0
                    7314150500379498593         0
40480049735387      5685690139879409547         0
                    597424223456586363          0
                    8841625760168847319         0
46479832058031      3613783563199627217         0
                    5685690139879409547         0
                    7314150500379498593         0
48857306404523      1367177154073382718         0
                    6670526099037031245         0
50153023443529      2211060154630359130         0
                    3613783563199627217         0
                    768579787521575291          0
50156198040329      1909641874861640857         0
                    3613783563199627217         0
                    6670526099037031245         0
...                                           ...
9223333305889619135 1852600517265062354         0
                    1909641874861640857         0
                    3613783563199627217         0
                    768579787521575291          0
                    8841625760168847319         0
9223335771334020529 1909641874861640857         0
                    453525480366550911          0
                    4911723539855588624         0
9223338824192191693 768579787521575291          1
9223339674432588374 8841625760168847319         0
9223347096934779058 2211060154630359130         0
                    5066527928272238333         0
9223351358419350988 3613783563199627217         0
9223351764270311555 8841625760168847319         0
9223352063517690707 1852600517265062354         0
                    2871729383671301763         1
9223354933755916779 1909641874861640857         0
9223356530868126408 1909641874861640857         0
                    6670526099037031245         0
                    6693726201323251689         0
9223357827590849236 2211060154630359130         0
                    5685690139879409547         0
9223358947751079260 1367177154073382718         0
9223361258074308080 768579787521575291          0
9223363919042445410 3613783563199627217         0
9223364994406019376 6670526099037031245         0
9223366924189890142 8769426218101861255         0
9223371754065880392 1909641874861640857         0
                    3613783563199627217         0
                    5685690139879409547         0

[5108312 rows x 1 columns]

In [7]:
data[['user_id','category_1','is_trade']].groupby(['user_id','category_1']).count()

is_trade
user_id             category_1                   
1523553916360       1909641874861640857         2
                    453525480366550911          2
                    6693726201323251689         4
5742370312099       2871729383671301763         2
6731361713351       8841625760168847319         1
8455785307263       8841625760168847319         4
9451887501191       768579787521575291          1
12217660235260      1852600517265062354         1
                    5685690139879409547         1
15213145006378      453525480366550911          1
                    768579787521575291          1
22634503310052      1909641874861640857         1
                    1916390345133212703         2
25684305694756      5685690139879409547         2
38014406741475      2871729383671301763         1
                    7314150500379498593         1
40480049735387      5685690139879409547         1
                    597424223456586363          1
                    8841625760168847319         1
46479832058031      3613783563199627217         1
                    5685690139879409547         1
                    7314150500379498593         2
48857306404523      1367177154073382718         2
                    6670526099037031245         1
50153023443529      2211060154630359130        13
                    3613783563199627217         3
                    768579787521575291          6
50156198040329      1909641874861640857         2
                    3613783563199627217         1
                    6670526099037031245         1
...                                           ...
9223333305889619135 1852600517265062354         6
                    1909641874861640857         3
                    3613783563199627217        11
                    768579787521575291          1
                    8841625760168847319         1
9223335771334020529 1909641874861640857         1
                    453525480366550911          2
                    4911723539855588624         1
9223338824192191693 768579787521575291          1
9223339674432588374 8841625760168847319         3
9223347096934779058 2211060154630359130         1
                    5066527928272238333         1
9223351358419350988 3613783563199627217         1
9223351764270311555 8841625760168847319         1
9223352063517690707 1852600517265062354         1
                    2871729383671301763         1
9223354933755916779 1909641874861640857         1
9223356530868126408 1909641874861640857         1
                    6670526099037031245         1
                    6693726201323251689         4
9223357827590849236 2211060154630359130         2
                    5685690139879409547         1
9223358947751079260 1367177154073382718         1
9223361258074308080 768579787521575291          1
9223363919042445410 3613783563199627217         2
9223364994406019376 6670526099037031245         3
9223366924189890142 8769426218101861255         1
9223371754065880392 1909641874861640857         7
                    3613783563199627217         2
                    5685690139879409547         1

[5108312 rows x 1 columns]

In [8]:
data[['user_id','category_1','is_trade']].groupby(['user_id','category_1']).sum()[data[['user_id','category_1','is_trade']].groupby(['user_id','category_1']).count()['is_trade']>10]

is_trade
user_id             category_1                   
50153023443529      2211060154630359130         0
503588114635900     1909641874861640857         0
894691882933622     8841625760168847319         0
967518836428444     768579787521575291          1
985250163814697     1852600517265062354         0
1340039722010437    1909641874861640857         1
1421994975205662    1909641874861640857         1
1894319204796277    453525480366550911          0
                    768579787521575291          0
2812034699042110    6693726201323251689         1
3025246849940958    3613783563199627217         0
3044585342583327    3613783563199627217         0
3147056199765819    8009556227083201357         0
3149521962559321    6693726201323251689         0
                    7314150500379498593         0
3529509790080158    3613783563199627217         0
3589038622761809    6670526099037031245         0
4199780669896963    5685690139879409547         0
4499357319220087    3348197449185791127         0
4579657058036800    3613783563199627217         0
4633474267582051    5685690139879409547         0
5093567217845118    1367177154073382718         0
5571713008332300    1909641874861640857         0
                    5685690139879409547         0
5776382299465672    6670526099037031245         1
5793600390465216    3613783563199627217         0
5877807711386663    6670526099037031245         0
5995769636112983    6693726201323251689         0
6359186094123341    1909641874861640857         0
                    3613783563199627217         0
...                                           ...
9218243574717682980 1852600517265062354         0
9218304039114314159 3613783563199627217         0
9218403575134639153 3613783563199627217         0
9218462506558815977 1909641874861640857         0
9218681816956401410 5685690139879409547         0
9218954083812318164 2211060154630359130         0
9219034511408476963 5685690139879409547         0
9219047478577001264 2211060154630359130         0
9219099973891960984 8841625760168847319         0
9219508425830353268 3613783563199627217         0
9219761117197423573 8841625760168847319         0
9219993880435493102 3613783563199627217         0
9220056463570064181 768579787521575291          1
9220250354184442035 1909641874861640857         0
9220628117730337288 5685690139879409547         0
9220843737699997805 5685690139879409547         0
9221402808334751819 3613783563199627217         0
9221405278507068334 4911723539855588624         0
9221967600783738210 2211060154630359130         0
9222029492673091498 3613783563199627217         0
9222031531539277979 394915394741096735          0
9222300517366990923 8841625760168847319         0
9222451545433578529 3613783563199627217         0
9222460807931109239 3613783563199627217         0
                    6670526099037031245         0
9222950188801714968 1909641874861640857         0
                    768579787521575291          0
9223002327918800743 3613783563199627217         0
9223049571495199700 3613783563199627217         0
9223333305889619135 3613783563199627217         0

[40739 rows x 1 columns]

## 开始写入字段

In [9]:
all_data = data.append(data2)
all_data = all_data.append(data3)

In [10]:
user_click = all_data[['user_id','category_1','instance_id']].groupby(['user_id','category_1']).count()

In [11]:
user_click = user_click.rename(columns=({'instance_id':'user_click'}))

In [12]:
user_click = user_click.reset_index()

In [13]:
user_click

,user_id,category_1,user_click
0,1523553916360,1909641874861640857,2
1,1523553916360,453525480366550911,2
2,1523553916360,6693726201323251689,4
3,5742370312099,2871729383671301763,2
4,6731361713351,8841625760168847319,1
5,8455785307263,8841625760168847319,4
6,9451887501191,768579787521575291,1
7,12217660235260,1852600517265062354,1
8,12217660235260,5685690139879409547,1
9,12885223301783,3434689896486063330,1


In [14]:
userclickdata = pd.merge(data,user_click,on=['user_id','category_1'],how='left')

In [15]:
userclickdata2 = pd.merge(data2,user_click,on=['user_id','category_1'],how='left')

In [16]:
userclickdata3 = pd.merge(data3,user_click,on=['user_id','category_1'],how='left')

In [17]:
userclickdata2

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,shop_score_service,shop_score_delivery,shop_score_description,category_0,category_1,category_2,property_0,property_1,property_2,user_click
0,93294255633855,2615021910672727866,3434999509929816124,196257267849351217,7,12,11,19,8447938236547496844,0,...,0.969971,0.969399,0.964373,836752724084922533,768579787521575291,,2636395404473730413,772325102984507165,1243768796732904674,12
1,558322259509454,3748469724871724205,2274097622688424543,3122721854741763495,10,11,12,21,1644660685156574751,0,...,0.983832,0.987976,0.989656,836752724084922533,5685690139879409547,7497531498747093265,6241534230954727302,367082587220462692,2072967855524022579,1
2,594573634113186,2111357467597282780,7838285046767229711,1019055478500227370,4,14,13,17,1899839434284249148,0,...,0.967112,0.967359,0.961470,836752724084922533,768579787521575291,,3540471119339706981,772325102984507165,251873273932348656,7
3,667327653735176,3717651022681022321,1962186990434139903,2174699138227015967,7,11,11,16,4108506449488241435,0,...,0.946592,0.944660,0.930338,836752724084922533,2211060154630359130,7848078693615386447,2072967855524022579,2636395404473730413,4621934203383159480,6
4,697732672924394,8284923347262844848,5390915051549817288,196257267849351217,6,12,12,18,1986129739312382401,0,...,0.977894,0.979554,0.981132,836752724084922533,1909641874861640857,,6241534230954727302,5977512434884267894,2072967855524022579,2
5,1467522731866334,122990815260714082,8334914790811141908,6219110439660858399,6,14,13,18,6545279318527448248,0,...,0.956073,0.955764,0.980872,836752724084922533,8841625760168847319,,6241534230954727302,367082587220462692,5977512434884267894,4
6,2564345731136577,5947872684354624624,7147251880558104750,6027631083130214337,8,12,11,19,6487822643782854186,0,...,0.977898,0.976857,0.984545,836752724084922533,2211060154630359130,7848078693615386447,6241534230954727302,367082587220462692,2072967855524022579,2
7,2635932702100660,4376223620063302249,969027688928478372,8835551476805511776,9,9,11,16,1162841664798642252,0,...,0.979576,0.977407,0.988806,836752724084922533,3613783563199627217,6370392357088946840,6241534230954727302,367082587220462692,2072967855524022579,3
8,3208825666465730,5226910727221958616,3601958674746901758,7534238860363577544,6,13,12,17,4693309019429064966,0,...,0.977467,0.972449,0.989476,836752724084922533,2871729383671301763,3492642177859571113,367082587220462692,5131280576272319091,8689439327673289769,6
9,3654424870386483,1981860414878829638,1637947898625411891,6219110439660858399,6,12,11,19,3155709470897336062,0,...,0.962937,0.963305,0.955644,836752724084922533,1916390345133212703,,6241534230954727302,367082587220462692,5977512434884267894,2


In [18]:
userclickdata[['instance_id','user_click']].to_csv('train_user_click.csv',index=False) 

In [19]:
userclickdata2[['instance_id','user_click']].to_csv('test_user_click.csv',index=False) 

In [20]:
userclickdata3[['instance_id','user_click']].to_csv('test1_user_click.csv',index=False) 

## 统计商品点击

In [21]:
data[['item_id','is_trade']].groupby(['item_id']).sum()

,is_trade
item_id,
42172734419742,10
119705681289325,0
139232509090699,0
186228696000288,0
208192717221960,0
271501162750665,4
334417922587028,0
345352588519597,3
412546248320450,0


In [22]:
data[['item_id','is_trade']].groupby(['item_id']).count()

,is_trade
item_id,
42172734419742,543
119705681289325,2
139232509090699,76
186228696000288,3
208192717221960,5
271501162750665,323
334417922587028,2
345352588519597,134
412546248320450,1


In [23]:
data[['item_id','is_trade']].groupby(['item_id']).sum()[data[['item_id','is_trade']].groupby(['item_id']).count().is_trade>200]

,is_trade
item_id,
42172734419742,10
271501162750665,4
1410182414862995,1
2135374266088353,36
3537019921042620,1
4238224537511743,4
4406891262180500,15
4691569635271766,4
5921857035436414,10


## 写入

In [24]:
item_click = all_data[['item_id','instance_id']].groupby(['item_id']).count()
item_click = item_click.rename(columns=({'instance_id':'item_click'}))

In [25]:
item_click

,item_click
item_id,
29268323426272,1
42172734419742,797
119705681289325,3
139232509090699,83
186228696000288,3
208192717221960,8
271501162750665,407
334417922587028,2
345352588519597,148


In [26]:
item_click = item_click.reset_index()

In [27]:
itemclickdata = pd.merge(data,item_click,on=['item_id'],how='left')
itemclickdata2 = pd.merge(data2,item_click,on=['item_id'],how='left')
itemclickdata3 = pd.merge(data3,item_click,on=['item_id'],how='left')

In [28]:
itemclickdata2

,instance_id,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_id,user_gender_id,...,shop_score_service,shop_score_delivery,shop_score_description,category_0,category_1,category_2,property_0,property_1,property_2,item_click
0,93294255633855,2615021910672727866,3434999509929816124,196257267849351217,7,12,11,19,8447938236547496844,0,...,0.969971,0.969399,0.964373,836752724084922533,768579787521575291,,2636395404473730413,772325102984507165,1243768796732904674,4682
1,558322259509454,3748469724871724205,2274097622688424543,3122721854741763495,10,11,12,21,1644660685156574751,0,...,0.983832,0.987976,0.989656,836752724084922533,5685690139879409547,7497531498747093265,6241534230954727302,367082587220462692,2072967855524022579,5380
2,594573634113186,2111357467597282780,7838285046767229711,1019055478500227370,4,14,13,17,1899839434284249148,0,...,0.967112,0.967359,0.961470,836752724084922533,768579787521575291,,3540471119339706981,772325102984507165,251873273932348656,698
3,667327653735176,3717651022681022321,1962186990434139903,2174699138227015967,7,11,11,16,4108506449488241435,0,...,0.946592,0.944660,0.930338,836752724084922533,2211060154630359130,7848078693615386447,2072967855524022579,2636395404473730413,4621934203383159480,906
4,697732672924394,8284923347262844848,5390915051549817288,196257267849351217,6,12,12,18,1986129739312382401,0,...,0.977894,0.979554,0.981132,836752724084922533,1909641874861640857,,6241534230954727302,5977512434884267894,2072967855524022579,2245
5,1467522731866334,122990815260714082,8334914790811141908,6219110439660858399,6,14,13,18,6545279318527448248,0,...,0.956073,0.955764,0.980872,836752724084922533,8841625760168847319,,6241534230954727302,367082587220462692,5977512434884267894,1807
6,2564345731136577,5947872684354624624,7147251880558104750,6027631083130214337,8,12,11,19,6487822643782854186,0,...,0.977898,0.976857,0.984545,836752724084922533,2211060154630359130,7848078693615386447,6241534230954727302,367082587220462692,2072967855524022579,3420
7,2635932702100660,4376223620063302249,969027688928478372,8835551476805511776,9,9,11,16,1162841664798642252,0,...,0.979576,0.977407,0.988806,836752724084922533,3613783563199627217,6370392357088946840,6241534230954727302,367082587220462692,2072967855524022579,471
8,3208825666465730,5226910727221958616,3601958674746901758,7534238860363577544,6,13,12,17,4693309019429064966,0,...,0.977467,0.972449,0.989476,836752724084922533,2871729383671301763,3492642177859571113,367082587220462692,5131280576272319091,8689439327673289769,1278
9,3654424870386483,1981860414878829638,1637947898625411891,6219110439660858399,6,12,11,19,3155709470897336062,0,...,0.962937,0.963305,0.955644,836752724084922533,1916390345133212703,,6241534230954727302,367082587220462692,5977512434884267894,4204


In [29]:
itemclickdata[['instance_id','item_click']].to_csv('train_item_click.csv',index=False) 
itemclickdata2[['instance_id','item_click']].to_csv('test_item_click.csv',index=False)
itemclickdata3[['instance_id','item_click']].to_csv('test1_item_click.csv',index=False) 

## 统计基于属性的点击

In [30]:
data[['user_id','property_0','is_trade']].groupby(['user_id','property_0']).sum()

is_trade
user_id             property_0                   
1523553916360       2072967855524022579         0
                    5977512434884267894         0
                    772325102984507165          0
5742370312099       367082587220462692          0
                    6241534230954727302         0
6731361713351       5977512434884267894         0
8455785307263       6241534230954727302         0
9451887501191       6241534230954727302         0
12217660235260      6241534230954727302         0
15213145006378      422690147597355277          0
                    6241534230954727302         0
22634503310052      367082587220462692          0
                    5977512434884267894         0
                    6241534230954727302         0
25684305694756      367082587220462692          0
                    6241534230954727302         0
38014406741475      6241534230954727302         0
                    629079007122254373          0
40480049735387      367082587220462692          0
                    6241534230954727302         0
46479832058031      2636395404473730413         0
                    6241534230954727302         0
                    7126426653086863522         0
48857306404523      6241534230954727302         0
                    9011444777478954178         0
50153023443529      2072967855524022579         0
                    2342428088144567289         0
                    367082587220462692          0
                    5977512434884267894         0
                    6241534230954727302         0
...                                           ...
9223333305889619135 367082587220462692          0
                    5977512434884267894         0
                    6241534230954727302         0
9223335771334020529 367082587220462692          0
                    6241534230954727302         0
                    7344985833148694227         0
9223338824192191693 6241534230954727302         1
9223339674432588374 2636395404473730413         0
                    6241534230954727302         0
9223347096934779058 2636395404473730413         0
                    5131280576272319091         0
9223351358419350988 2072967855524022579         0
9223351764270311555 5977512434884267894         0
9223352063517690707 6241534230954727302         1
9223354933755916779 2072967855524022579         0
9223356530868126408 4621934203383159480         0
                    6241534230954727302         0
                    772325102984507165          0
9223357827590849236 6241534230954727302         0
9223358947751079260 6241534230954727302         0
9223361258074308080 367082587220462692          0
9223363919042445410 3995998797710394988         0
                    6241534230954727302         0
9223364994406019376 5131280576272319091         0
                    6241534230954727302         0
9223366924189890142 2072967855524022579         0
9223371754065880392 2072967855524022579         0
                    2636395404473730413         0
                    367082587220462692          0
                    6241534230954727302         0

[5706167 rows x 1 columns]

In [31]:
data[['user_id','property_1','is_trade']].groupby(['user_id','property_1']).sum()

is_trade
user_id             property_1                   
1523553916360       143660112368942257          0
                    18773227022480660           0
                    3145741777362890024         0
                    3163265386149801264         0
                    4621934203383159480         0
                    8851425427477388664         0
5742370312099       367082587220462692          0
                    5131280576272319091         0
6731361713351       5131280576272319091         0
8455785307263       367082587220462692          0
                    5977512434884267894         0
9451887501191       367082587220462692          0
12217660235260      2072967855524022579         0
                    367082587220462692          0
15213145006378      3995998797710394988         0
                    4067341101015777832         0
22634503310052      18773227022480660           0
                    2072967855524022579         0
                    5977512434884267894         0
25684305694756      367082587220462692          0
                    5977512434884267894         0
38014406741475      367082587220462692          0
                    5131280576272319091         0
40480049735387      367082587220462692          0
                    5131280576272319091         0
                    5977512434884267894         0
46479832058031      2636395404473730413         0
                    367082587220462692          0
                    6641007758333551406         0
48857306404523      3657871859501171040         0
...                                           ...
9223333305889619135 5977512434884267894         0
9223335771334020529 2072967855524022579         0
                    2488701964620097331         0
                    367082587220462692          0
9223338824192191693 3356339000963149779         1
9223339674432588374 367082587220462692          0
                    4562747919324019386         0
9223347096934779058 4621934203383159480         0
                    9148482949976129397         0
9223351358419350988 2072967855524022579         0
9223351764270311555 2636395404473730413         0
9223352063517690707 367082587220462692          1
9223354933755916779 2636395404473730413         0
9223356530868126408 2072967855524022579         0
                    3163265386149801264         0
                    4621934203383159480         0
                    5131280576272319091         0
9223357827590849236 367082587220462692          0
                    5131280576272319091         0
                    7344985833148694227         0
9223358947751079260 367082587220462692          0
9223361258074308080 3540471119339706981         0
9223363919042445410 5977512434884267894         0
9223364994406019376 367082587220462692          0
                    9148482949976129397         0
9223366924189890142 7839592306500064003         0
9223371754065880392 2072967855524022579         0
                    2636395404473730413         0
                    367082587220462692          0
                    772325102984507165          0

[6750716 rows x 1 columns]

In [32]:
data[['user_id','property_2','is_trade']].groupby(['user_id','property_2']).sum()

is_trade
user_id             property_2                   
1523553916360       3145741777362890024         0
                    3163265386149801264         0
                    6641007758333551406         0
                    7344985833148694227         0
                    9011444777478954178         0
                    9148482949976129397         0
5742370312099       5131280576272319091         0
                    8689439327673289769         0
6731361713351       422690147597355277          0
8455785307263       2072967855524022579         0
                    5977512434884267894         0
9451887501191       2072967855524022579         0
12217660235260      5977512434884267894         0
                    8218883658021718578         0
15213145006378      3540471119339706981         0
                    6523907184013144830         0
22634503310052      2072967855524022579         0
                    2636395404473730413         0
                    7839592306500064003         0
25684305694756      5131280576272319091         0
38014406741475      1095835400823143663         0
                    5791066974214309919         0
40480049735387      2072967855524022579         0
                    2636395404473730413         0
46479832058031      2072967855524022579         0
                    2636395404473730413         0
                    4038060334629950706         0
                    9148482949976129397         0
48857306404523      2891682021882519203         0
                    3995998797710394988         0
...                                           ...
9223338824192191693 3540471119339706981         1
9223339674432588374 5977512434884267894         0
                    7839592306500064003         0
9223347096934779058 3145741777362890024         0
                    792781902817689193          0
9223351358419350988 3163265386149801264         0
9223351764270311555 7344985833148694227         0
9223352063517690707 2072967855524022579         0
                    8689439327673289769         1
9223354933755916779 655790194507223273          0
9223356530868126408 2072967855524022579         0
                    7344985833148694227         0
                    7515802706813700848         0
                    8399888477341455089         0
9223357827590849236 2636395404473730413         0
                    8070395809181082496         0
                    8851425427477388664         0
9223358947751079260 3995998797710394988         0
9223361258074308080 7138596973623795131         0
9223363919042445410 2072967855524022579         0
                    2636395404473730413         0
9223364994406019376 3502274680676470611         0
                    3995998797710394988         0
                    4875993958863530301         0
9223366924189890142 9148482949976129397         0
9223371754065880392 2072967855524022579         0
                    2636395404473730413         0
                    5977512434884267894         0
                    7839592306500064003         0
                    8689439327673289769         0

[7541367 rows x 1 columns]

In [33]:
d1 = all_data[['user_id','property_0','instance_id']].groupby(['user_id','property_0']).count().reset_index()

In [34]:
d1 = d1.rename(columns=({'instance_id':'property_0_click'}))

In [35]:
d2 = all_data[['user_id','property_1','instance_id']].groupby(['user_id','property_1']).count().reset_index()

In [36]:
d2 = d2.rename(columns=({'instance_id':'property_1_click'}))

In [37]:
d3 = all_data[['user_id','property_2','instance_id']].groupby(['user_id','property_2']).count().reset_index()

In [38]:
d3 = d3.rename(columns=({'instance_id':'property_2_click'}))

In [39]:
d123 = pd.merge(data[['instance_id','user_id','property_0','property_1','property_2']],d1,on=['user_id','property_0'],how='left')

In [40]:
d123 = pd.merge(d123,d2,on=['user_id','property_1'],how='left')

In [41]:
d123 = pd.merge(d123,d3,on=['user_id','property_2'],how='left')

In [42]:
del d123['user_id']
del d123['property_0']
del d123['property_1']
del d123['property_2']

In [43]:
d123

,instance_id,property_0_click,property_1_click,property_2_click
0,7118167540371,1,1,5
1,43508800019098,1,1,1
2,78067858668749,2,1,1
3,137525720365644,1,1,1
4,151193932520734,1,1,1
5,191034067765499,1,1,1
6,221786632908010,1,1,1
7,233375766178961,3,2,1
8,235542268576836,4,4,2
9,271028956022181,1,1,1


In [44]:
d123.to_csv('train_property_click.csv',index=False) 

In [45]:
t1 = all_data[['user_id','property_0','instance_id']].groupby(['user_id','property_0']).count().reset_index()
t1 = t1.rename(columns=({'instance_id':'property_0_click'}))
t2 = all_data[['user_id','property_1','instance_id']].groupby(['user_id','property_1']).count().reset_index()
t2 = t2.rename(columns=({'instance_id':'property_1_click'}))
t3 = all_data[['user_id','property_2','instance_id']].groupby(['user_id','property_2']).count().reset_index()
t3 = t3.rename(columns=({'instance_id':'property_2_click'}))
t123 = pd.merge(data2[['instance_id','user_id','property_0','property_1','property_2']],t1,on=['user_id','property_0'],how='left')
t123 = pd.merge(t123,t2,on=['user_id','property_1'],how='left')
t123 = pd.merge(t123,t3,on=['user_id','property_2'],how='left')
del t123['user_id']
del t123['property_0']
del t123['property_1']
del t123['property_2']

In [46]:
t123

,instance_id,property_0_click,property_1_click,property_2_click
0,93294255633855,1,1,2
1,558322259509454,1,1,1
2,594573634113186,6,3,1
3,667327653735176,3,3,3
4,697732672924394,3,1,3
5,1467522731866334,12,12,8
6,2564345731136577,2,2,2
7,2635932702100660,2,2,2
8,3208825666465730,3,2,3
9,3654424870386483,2,2,2


In [47]:
t123.to_csv('test_property_click.csv',index=False) 

In [48]:
t1 = all_data[['user_id','property_0','instance_id']].groupby(['user_id','property_0']).count().reset_index()
t1 = t1.rename(columns=({'instance_id':'property_0_click'}))
t2 = all_data[['user_id','property_1','instance_id']].groupby(['user_id','property_1']).count().reset_index()
t2 = t2.rename(columns=({'instance_id':'property_1_click'}))
t3 = all_data[['user_id','property_2','instance_id']].groupby(['user_id','property_2']).count().reset_index()
t3 = t3.rename(columns=({'instance_id':'property_2_click'}))
t123 = pd.merge(data3[['instance_id','user_id','property_0','property_1','property_2']],t1,on=['user_id','property_0'],how='left')
t123 = pd.merge(t123,t2,on=['user_id','property_1'],how='left')
t123 = pd.merge(t123,t3,on=['user_id','property_2'],how='left')
del t123['user_id']
del t123['property_0']
del t123['property_1']
del t123['property_2']

In [49]:
t123.to_csv('test1_property_click.csv',index=False) 

In [50]:
data[['shop_id','instance_id']].groupby(['shop_id']).count()

,instance_id
shop_id,
369399843124909,142
1149611804947766,3
1394879586952028,276
1521332146149762,15
2152308100603334,8
2773765767290054,88
3264820913535821,137
4131379046539124,437
4267495697515624,277


In [51]:
s1 = all_data[['shop_id','instance_id']].groupby(['shop_id']).count().reset_index()
s1 = s1.rename(columns=({'instance_id':'shop_click'}))

In [52]:
s1

,shop_id,shop_click
0,369399843124909,160
1,1149611804947766,4
2,1394879586952028,311
3,1521332146149762,31
4,2152308100603334,8
5,2773765767290054,125
6,3264820913535821,158
7,4131379046539124,538
8,4267495697515624,315
9,4299863091308684,16


In [53]:
s123 = pd.merge(data[['instance_id','shop_id']],s1,on=['shop_id'],how='left')

In [54]:
s456 = pd.merge(data2[['instance_id','shop_id']],s1,on=['shop_id'],how='left')

In [55]:
del s123['shop_id']
del s456['shop_id']

In [56]:
s123.to_csv('train_shop_id_click.csv',index=False) 

In [57]:
s456.to_csv('test_shop_id_click.csv',index=False) 

In [58]:
s456 = pd.merge(data3[['instance_id','shop_id']],s1,on=['shop_id'],how='left')
del s456['shop_id']

In [59]:
s456.to_csv('test1_shop_id_click.csv',index=False) 

In [60]:
s123.shape

(10432036, 2)

In [61]:
b1 = all_data[['item_brand_id','instance_id']].groupby(['item_brand_id']).count().reset_index()
b1 = b1.rename(columns=({'instance_id':'brand_click'}))

In [62]:
b123 = pd.merge(data[['instance_id','item_brand_id']],b1,on=['item_brand_id'],how='left')

In [63]:
b456 = pd.merge(data2[['instance_id','item_brand_id']],b1,on=['item_brand_id'],how='left')

In [64]:
del b123['item_brand_id']
del b456['item_brand_id']


In [65]:
b456.shape

(519888, 2)

In [66]:
b123.to_csv('train_brand_click.csv',index=False) 
b456.to_csv('test_brand_click.csv',index=False) 

In [67]:
b456 = pd.merge(data3[['instance_id','item_brand_id']],b1,on=['item_brand_id'],how='left')

In [68]:
b456.shape

(1209768, 3)

In [69]:
del b456['item_brand_id']

In [70]:
b456.to_csv('test1_brand_click.csv',index=False) 